In [3]:
import json
mimic_train_path = "/data/aofei/hallucination/RULE/training/alignment/mimic.json"
with open(mimic_train_path, "r") as f:
    mimic_train = json.load(f)
len(mimic_train)

iuxray_train_path = "/data/aofei/hallucination/RULE/training/alignment/iuxray.json"
with open(iuxray_train_path, "r") as f:
    iuxray_train = json.load(f)
len(iuxray_train)

iuxray_test_path = "/data/aofei/hallucination/RULE/test/iuxray_test.jsonl"
with open(iuxray_test_path, 'r') as file:
    iuxray_test = [json.loads(line) for line in file]
len(iuxray_test)

2573

In [4]:
iuxray_train[0]

{'id': 'CXR2079_IM-0711/0.png',
 'image': 'CXR2079_IM-0711/0.png',
 'conversations': [{'from': 'human',
   'value': 'You are provided with a chest X-ray image, a image-related question and 1 reference report(s): Stable cardiomediastinal silhouette. Stable XXXX opacity in the left base, XXXX scarring or atelectasis. Rounded calcified density in the left lung base, XXXX calcified granuloma. No XXXX consolidation. No pleural effusion or pneumothorax. Stable degenerative changes of the spine.\nPlease answer the question based on the image and report and choose from the following two options: [yes, no]. It should be noted that the diagnostic information in the reference reports cannot be directly used as the basis for diagnosis, but should only be used for reference and comparison. Question: Is there any focal airspace opacity present?\n<image>'},
  {'from': 'gpt',
   'value': 'The chest x-ray does not show any focal airspace opacity.'}],
 'rejected_conversations': [{'from': 'human',
   'va

In [9]:
new_iuxray_data = []
_id = 0
for i in iuxray_train:
    template = dict()
    template['conversations'] = []
    template['image'] = i['image']
    template['id'] = _id
    old_conv = i['conversations']
    query = "<image>\n" + old_conv[0]['value'].split("Question:")[-1].strip().replace("\n<image>", "")
    ans = old_conv[1]['value']
    new_qa = {"from": "human", "value": query}
    new_qa2 = {"from": "gpt", "value": ans}
    template['conversations'] += [new_qa, new_qa2]
    new_iuxray_data.append(template)
    _id += 1

new_iuxray_data[-1]

{'conversations': [{'from': 'human',
   'value': '<image>\nAre there scattered calcified granulomas seen on the chest X-ray?'},
  {'from': 'gpt',
   'value': 'Yes, the chest X-ray shows scattered calcified granulomas.'}],
 'image': 'CXR729_IM-2288/0.png',
 'id': 1578}

In [10]:
with open(r"/data/aofei/hallucination/IU_Xray/data/training.json", "w") as f:
    json.dump(new_iuxray_data, f, indent=4)

In [5]:
iuxray_test[0]

{'question': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? \n<image>',
 'answer': 'Yes.',
 'image': 'CXR3030_IM-1405/0.png',
 'question_id': 1,
 'text': 'Does the cardiomediastinal silhouette appear normal in the chest X-ray? Please choose from the following two options: [yes, no]\n<image>',
 'report': 'Normal cardiomediastinal silhouette. There is no focal consolidation. There are no XXXX of a large pleural effusion. There is no pneumothorax. There is no acute bony abnormality seen.'}

In [6]:
new_iuxray_test_data = []

for i in iuxray_test:
    template = dict()
    template['conversations'] = []
    template['image'] = i['image']
    template['id'] = i['question_id']
    query = "<image>\n" + i['question'].strip().replace("\n<image>", "")
    ans = i['answer']
    template["answer_type"] = "CLOSED"
    new_qa = {"from": "human", "value": query}
    new_qa2 = {"from": "gpt", "value": ans}
    template['conversations'] += [new_qa, new_qa2]
    new_iuxray_test_data.append(template)

new_iuxray_test_data[-1]

{'conversations': [{'from': 'human',
   'value': '<image>\nAre the mediastinal contours within normal limits? '},
  {'from': 'gpt',
   'value': 'Yes, since the report does not mention any abnormalities of the mediastinum.'}],
 'image': 'CXR49_IM-2110/0.png',
 'id': 2573,
 'answer_type': 'CLOSED'}

In [7]:
len(new_iuxray_test_data)

2573

In [8]:
with open(r"/data/aofei/hallucination/IU_Xray/data/test.json", "w") as f:
    json.dump(new_iuxray_test_data, f, indent=4)